# Analytical Parameter Model Optimizer using Tensorflow

### Import Statements

In [ ]:
import pandas as pd
import numpy as np
import random
from numpy.polynomial.hermite import hermgauss
from __future__ import print_function
import tensorflow as tf

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

### Initialize DataFrame to store data for EDA

In [ ]:
tf_parameters = ['step', 'uBA','sbA','uBB','sbB','pA','pb','pc', 'pm','umax','nl',
                 'eLJ','uc','cost']
df_params = pd.DataFrame(columns = tf_parameters)

### Read in Data

In [ ]:
bedam_data = pd.read_csv("sampl4-oah-b7eqnosc.repl.cycle.totE.potE.temp.lambda.ebind.dat",delim_whitespace=True, 
                          header=None,names=["replica","cycle","totE",
                                             "potE","temp","Lambda","ebind"])

#ulambda = pandas.read_csv("bcd-nabumetone.repl",delim_whitespace=True, 
#                          header=None,names=["replica","cycle","totE",
#                          "potE","temp","Lambda","ebind"])

In [ ]:
bedam_data.dtypes

In [ ]:
kT = 1.986e-3*300.0 # [kcal/mol]
beta = 1/kT
bedam_data['u'] = bedam_data['ebind']*beta
bedam_data.head()

In [ ]:
bedam_data['mask'] = np.logical_and(bedam_data['Lambda'] > 1.e-6, bedam_data['cycle'] > 100)
bedam_data.tail()

In [ ]:
rel_data = bedam_data.loc[bedam_data['mask'] == True]
rel_data.head()

In [ ]:
rel_data['u'] = rel_data['u'].astype(np.float32)
rel_data['Lambda'] = rel_data['Lambda'].astype(np.float32)
rel_data['U'] = rel_data['u']-1.e6*beta # u-umax
rel_data.dtypes

### Initialize parameters

parameters that are being optimized are stored in a `tf.Variable`, those that are kept constant are stored as `tf.constant`

In [ ]:
pb = 1.18e-4
pc = 3.00e-2
uc = 0.5
params = {'UBA':tf.constant(-13.953*beta),
          'SIGBA':tf.constant(1.906*beta),
          'UBB':tf.constant(-2.334*beta),
          'SIGBB':tf.constant(3.238*beta),
          'PA':tf.constant(2.45e-4),
          'E':tf.constant(20.0*beta),
          'UC':tf.constant(uc*beta),
          'NL':tf.Variable(2.96),
          'PB':tf.Variable(pb),
          'PC':tf.Variable(pc)}

pi = tf.constant(np.pi, dtype=tf.float32)

### Get variables and weights for Hermite-Gauss Approximation

Here we use 15 Hermite-Gauss nodes. They are placed symmetrically around zero. The 7 negative nodes are not used.

In [ ]:
x_gauss,w_gauss = hermgauss(15)
n_gauss = tf.constant(x_gauss.size, dtype=tf.int32)
x_gauss = tf.constant(x_gauss, dtype=tf.float32)
w_gauss = tf.constant(w_gauss, dtype=tf.float32)

### Load (u,lambda) data pairs in constant tensors


In [ ]:
u = tf.constant(rel_data['u'])
lambdas = tf.constant(rel_data['Lambda'])


## Definition of Likelihood function in TensorFlow

The negative of the log-likelihood function is:

$-\mathcal{\ln L}(\theta)=-\sum_{i}\ln p_{\lambda_{i}}(u_{i}|\theta)=-\sum_{i}\ln [ e^{-\lambda_{i}u_{i}} p_{0}(u_{i}|\theta)/K(\lambda_{i}|\theta) ]$

where $\theta$ represents the parameters,

$p_{0}(u)=p_{b}p_B(u) + p_{m}p_M(u)+p_{{\rm c}}\int_{\tilde{u}_{C}}^{+\infty}p_{WCA}(u')p_B(u-u')u'$

$K(\lambda)=K_{C}(\lambda)K_{B}(\lambda)$

$K_{C}(\lambda)= p_{b}+p_{m}e^{-\lambda u_{{\rm max}}}+p_{c}K_{WCA}(\lambda)$

$K{}_{WCA}(\lambda)=\int_{\tilde{u}_{C}}^{u_{{\rm max}}}p_{WCA}(u)e^{-\lambda u}du$

$K_{B}(\lambda)=\int_{-\infty}^{+\infty}p_{B}(u)e^{-\lambda u}=e^{\sigma_{B}^{2}\lambda(\lambda/2-\bar{u}_{B}/\sigma_{B}^{2})}$



### Gaussian mixture for B for all binding energies

This computes $p_B(u)$ for all binding energy samples. In general $p_B(u)$ is a mixture of two (or more) Gaussians.

gauss_bA = $\frac{1}{\sqrt{2\pi}\sigma_{BA}}e^{\frac{-(u-u_{BA})^2}{2\sigma_{BA}^2}}$ 
<br>
gauss_bB = $\frac{1}{\sqrt{2\pi}\sigma_{BB}}e^{\frac{-(u-u_{BB})^2}{2\sigma_{BB}^2}}$ 
<br>
gauss_b = $P_A *\frac{1}{\sqrt{2\pi}\sigma_{BA}}e^{\frac{-(u-u_{BA})^2}{2\sigma_{BA}^2}} + (1-P_A)*\frac{1}{\sqrt{2\pi}\sigma_{BB}}e^{\frac{-(u-u_{BB})^2}{2\sigma_{BB}^2}}$

where $P_A$ and $P_B = 1 - P_A$ are the populations of the two conformational states at $\lambda = 0$

In [ ]:
def gaussian(params,pi,u):
    
    gauss_bA = tf.exp(-tf.pow(u-params['UBA'],2)/(2.0*tf.pow(params['SIGBA'],2)))/(tf.sqrt(2.*pi)*params['SIGBA'])
    gauss_bB = tf.exp(-tf.pow(u-params['UBB'],2)/(2.0*tf.pow(params['SIGBB'],2)))/(tf.sqrt(2.*pi)*params['SIGBB'])
    gauss_b = params['PA']*gauss_bA + gauss_bB - params['PA']*gauss_bB
    
    return gauss_b

### Gaussian for energies at large distances where U -> u-umax where umax = $10^6/kT$

this computes $p_M(u)$, which has the same form and parameters of $p_B(u)$ but is centered at $\bar{u}_B + u_{\rm max}$ 

gauss_bA_far = $\frac{1}{\sqrt{2\pi}\sigma_{BA}}e^{\frac{-(U-u_{BA})^2}{2\sigma_{BA}^2}}$ 
<br>
gauss_bB_far = $\frac{1}{\sqrt{2\pi}\sigma_{BB}}e^{\frac{-(U-u_{BB})^2}{2\sigma_{BB}^2}}$ 
<br>
gauss_b_far = $P_A *\frac{1}{\sqrt{2\pi}\sigma_{BA}}e^{\frac{-(U-u_{BA})^2}{2\sigma_{BA}^2}} + (1-P_A)*\frac{1}{\sqrt{2\pi}\sigma_{BB}}e^{\frac{-(U-u_{BB})^2}{2\sigma_{BB}^2}}$

U is $u - u_{\rm max}$ so the argument of each Gaussian is $u - (u_{\rm max} +  \bar{u}_B)$.

In [ ]:
U = tf.constant(rel_data['U'])

def gaussian_far(params,pi,U):
    
    gauss_bA_far = tf.exp(-tf.pow(U-params['UBA'],2)/(2.0*tf.pow(params['SIGBA'],2)))/(tf.sqrt(2.*pi)*params['SIGBA'])
    gauss_bB_far = tf.exp(-tf.pow(U-params['UBB'],2)/(2.0*tf.pow(params['SIGBB'],2)))/(tf.sqrt(2.*pi)*params['SIGBB'])
    gauss_b_far = params['PA']*gauss_bA_far + gauss_bB_far - params['PA']*gauss_bB_far
    
    return gauss_b_far

### Convolution of gaussian (mixture) and pwca

pwca = $n_L * \left[1-\frac{\sqrt{1+\hat{x}_c}}{\sqrt{1+x}} \right]^{n_L-1}*\frac{Heaviside(u_c-\hat{u}_c)\sqrt{1+\hat{x}_c}}{4\epsilon_{LJ}x(1+x)^{3/2}}$ <br>
where <br>
$x = \sqrt{u_c/\epsilon_{LJ}}$ <br>
$\hat{x}_c = \sqrt{\hat{u}_c/\epsilon_{LJ}}$

#### Assuming statistical independence of the background and collisional energies, the probability density of the total binding energy is given by $P_0$ which is the convolution of their respective probability densities

$p_0(u) = p_b p_B(u) + p_m p_M(u) + p_c \int_{u_c}^\infty p_{wca}(u')p_B(u-u')du'$

`nm` is an approximation such that to the normalization factor of $p_{wca}$ between $u_C$ and $u_{\rm max}$ is ${\rm nm} \times nl$ 

In [ ]:
umax = 1.e6*beta
pm = tf.constant(1. - pb - pc)
xc = tf.sqrt(params['UC']/params['E'])
a = tf.sqrt(1.+xc)
eps = xc/10.
xm = tf.sqrt(umax/params['E'])
nm = tf.pow(1. - params['NL']*a/xm, -1)


this performs the convolution. Once for each of the two states (A and B) of the mixture in $p_B(u)$

In [ ]:
gauss_b = gaussian(params,pi,u)
gauss_b_far = gaussian_far(params,pi,U)
sq2 = tf.sqrt(2.)

def pwcaA(params,u,x_gauss,eps,nm):
    """ Pwca for all input y's """
    sq2 = tf.sqrt(2.)
    yA = sq2*params['SIGBA']*x_gauss + u[:,None] - params['UBA']
    x1A = tf.pow(yA/params['E'],2) #to make x positive
    xA  = tf.pow( x1A , 0.25 )
    bA = tf.sqrt(1.+ xA)
    z1A = tf.tanh( tf.pow(a/bA,12.) )
    zA = tf.pow( z1A, 1./12. ) #caps z to 1
    fcore2A = tf.pow((1.-zA), params['NL']-1.)
    fcore3A = a/((xA+eps)*tf.pow(bA,3)*4.*params['E'])
    fcore4A = tf.sigmoid(20.*(yA-0.5*params['UC'])/params['UC'])
    pwcaA = nm*params['NL']*fcore2A*fcore3A*fcore4A
    
    return pwcaA
#---
pwca_A = pwcaA(params,u,x_gauss,eps,nm)
qA = tf.matmul(pwca_A,tf.reshape(w_gauss,[n_gauss,1]))/tf.sqrt(pi)
q2A = qA[:,0]

def pwcaB(params,u,x_gauss,eps,nm):

    yB = sq2*params['SIGBB']*x_gauss + u[:,None] - params['UBB']
    x1B = tf.pow(yB/params['E'],2) #to make x positive
    xB  = tf.pow( x1B , 0.25 )
    bB = tf.sqrt(1.+ xB)
    z1B = tf.tanh( tf.pow(a/bB,12.) )
    zB = tf.pow( z1B, 1./12. ) #caps z to 1
    fcore2B = tf.pow((1.-zB), params['NL']-1.)
    fcore3B = a/((xB+eps)*tf.pow(bB,3)*4.*params['E'])
    fcore4B = tf.sigmoid(20.*(yB-0.5*params['UC'])/params['UC'])
    pwcaB = nm*params['NL']*fcore2B*fcore3B*fcore4B
    
    return pwcaB
#---
pwca_B = pwcaB(params,u,x_gauss,eps,nm)

qB = tf.matmul(pwca_B,tf.reshape(w_gauss,[n_gauss,1]))/tf.sqrt(pi)
q2B = qB[:,0]

q2 = params['PA']*q2A + q2B - params['PA']*q2B

#p0's
p0 = params['PB']*gauss_b + params['PC']*q2 + pm*gauss_b_far



### Model for the free energy profile

this is $K_{B}(\lambda)=\int_{-\infty}^{+\infty}p_{B}(u)e^{-\lambda u}=e^{\sigma_{B}^{2}\lambda(\lambda/2-\bar{u}_{B}/\sigma_{B}^{2})}$ but for the state mixture

In [ ]:
#kB1
klBA = tf.exp(0.5*tf.pow(params['SIGBA'],2)*tf.pow(lambdas,2) - lambdas*params['UBA'])
klBB = tf.exp(0.5*tf.pow(params['SIGBB'],2)*tf.pow(lambdas,2) - lambdas*params['UBB']) 
klB1 = params['PA']*klBA + klBB - params['PA']*klBB



this sets up an exponential grid to perform

$K{}_{WCA}(\lambda)=\int_{\tilde{u}_{C}}^{u_{{\rm max}}}p_{WCA}(u)e^{-\lambda u}du$

`xasymp` is the actual grid ($x$), `dxasymp` is the grid spacing ($dx$)

In [ ]:
a = uc*beta
umax = 1.e6*beta
ymax = np.log(umax)/a 
    
def grid():
    x_grid = {}
    #a = uc*beta
    #umax = 1.e6*beta
    #ymax = np.log(umax)/a # 
    dy = ymax/100.0
    xasymp = np.float32(np.exp(a*np.arange(0.,ymax,dy)) - 1.) + 1.e-6
    dxasymp = np.float32(xasymp[1:] - xasymp[:-1])
    xasymp = np.float32(xasymp[:-1])

    x_grid['nx'] = tf.constant(dxasymp.size, dtype=tf.int32)
    x_grid['xasymp'] = tf.constant(xasymp, dtype=tf.float32)
    x_grid['dxasymp'] = tf.constant(dxasymp, dtype=tf.float32)
    
    return x_grid


this computes a modified $p_{wca}(u)$ over the integration grid to avoid the step function:

$p_{WCA}(u_{C})\simeq n_{l}\left[1-\tanh(z^{12})^{1/12}\right]^{n_{l}-1}\frac{s(u_{C}-\tilde{u}_{C})}{4\epsilon_{LJ}}\frac{(1+x_{C})^{1/2}}{x(1+x)^{3/2}}$

where $z=(1+x_{C})^{1/2}/(1+x)^{1/2}$ and $s(u)=[1+\exp(-20u/\tilde{u}_{C})]^{-1}$ is the sigmoid function.

It then perform the integration for all possible $\lambda$ values by computing a 2-D $\exp(-\lambda u)$ tensor and performing a matrix multiplication.

In [ ]:
x_grid = grid()
def pwca_x(params,x_grid):
    """ pwca for the x grid """

    x1_s = tf.pow(x_grid['xasymp']/params['E'],2) #to make x positive
    x_s  = tf.pow( x1_s , 0.25 )
    b_s = tf.sqrt(1.+ x_s)
    z1_s = tf.tanh( tf.pow(a/b_s,12.) )
    z_s = tf.pow( z1_s, 1./12. ) #caps z to 1
    fcore2_s = tf.pow((1.-z_s), params['NL']-1.)
    fcore3_s = a/((x_s+eps)*tf.pow(b_s,3)*4.*params['E'])
    fcore4_s = tf.sigmoid(20.*(x_grid['xasymp']-0.5*params['UC'])/params['UC'])
    pwca_s = nm*params['NL']*fcore2_s*fcore3_s*fcore4_s
    
    return pwca_s
pwca_s = pwca_x(params,x_grid)

def kwca(x_grid,pwca_s):
    #kwca
    fsamples = x_grid['dxasymp']*pwca_s
    expl = tf.exp(- x_grid['xasymp'] * lambdas[:,None])
    q_C = tf.matmul(expl,tf.reshape(fsamples,[x_grid['nx'],1]))
    klwca = q_C[:,0]
    return klwca

klwca= kwca(x_grid,pwca_s)


Finally it assembles $p_0(u)$ and $K(\lambda)$ for each data point into the -log likelihood.

In [ ]:
#free energies
e2 = tf.exp(-1.e6*beta * lambdas)
klC = params['PB'] + e2 - params['PC']*e2 - params['PB']*e2 + params['PC'] * klwca
kl = klB1*klC
pkl = p0/kl

#cost function
cost = -tf.reduce_sum(tf.log(pkl))

### Initialize and start tensorflow optimizer session

In [ ]:
optimizer = tf.train.AdamOptimizer(2.e-4)

train = optimizer.minimize(cost)

init = tf.global_variables_initializer()

gradient_cost = optimizer.compute_gradients(cost)


with tf.Session() as sess:
    
    sess.run(init)
    ll = sess.run(cost)

    best_loss = ll
    best_ubA = sess.run(params['UBA'])
    best_sbA = sess.run(params['SIGBA'])
    best_ubB = sess.run(params['UBB'])
    best_sbB = sess.run(params['SIGBB'])
    best_pA = sess.run(params['PA'])
    best_pb = sess.run(params['PB'])
    best_pc = sess.run(params['PC'])
    best_nl = sess.run(params['NL'])
    best_elj = sess.run(params['E'])
    best_uc = sess.run(params['UC'])   
    for step in range(100):
        sess.run(train)
        ll = sess.run(cost)
        lubA = sess.run(params['UBA'])
        lsbA = sess.run(params['SIGBA'])
        lubB = sess.run(params['UBB'])
        lsbB = sess.run(params['SIGBB'])
        lpA =  sess.run(params['PA'])
        lpb = sess.run(params['PB'])
        lpc = sess.run(params['PC'])
        lnl = sess.run(params['NL'])
        lelj = sess.run(params['E'])
        luc = sess.run(params['UC'])                     
        if( ll < best_loss ):
            best_loss = ll
            best_ubA = lubA
            best_sbA = lsbA
            best_ubB = lubB
            best_sbB = lsbB
            best_pA = lpA
            best_pb = lpb
            best_pc = lpc
            best_nl = lnl
            best_elj = lelj
            best_uc = luc
      
        df_params = df_params.append({'step':step,'uBA':lubA*kT,'sbA':lsbA*kT,
                                           'uBB':lubB*kT,'sbB':lsbB*kT,'pA':lpA,'pb':lpb,
                                           'pc':lpc,'pm':1.0-pb-pc,'umax':umax,'nl':lnl,'eLJ':lelj*kT,
                                           'uc':luc*kT,'cost':ll},ignore_index=True)

    print("----- End of optimization --------");
    print("best", "x:", " uBA =", best_ubA*kT, "sbA =", best_sbA*kT, "uBB =", best_ubB*kT, "sbB =", best_sbB*kT, "pA =", best_pA, "pb = ", best_pb, "pc =", best_pc, "pm = ", 1.0-pb-pc, "umax = ", umax, "nl = ", best_nl, "eLJ = ", best_elj*kT, "uc = ", best_uc*kT, "cost =", best_loss)



In [ ]:
df_params.head()

In [ ]:
df_params.tail()

In [ ]:
df_params.describe()

In [ ]:
df_cost = df_params[['step','cost']]
df_cost.tail()

In [ ]:
fig = plt.figure()

ax0 = fig.add_subplot(221)
ax1 = fig.add_subplot(222)
ax2 = fig.add_subplot(223)
ax3 = fig.add_subplot(224)

# 1st plot
df_params.plot(kind='line',x='step',y='cost', figsize=(20,10),ax=ax0)
ax0.set_ylabel("Cost")
ax0.set_xlabel("Steps")

# 2nd plot
df_params.plot(kind='line',x='step',y='nl',figsize=(20,10),ax=ax1)
ax1.set_ylabel("NL")
ax1.set_xlabel("Steps")

# 3rd plot
df_params.plot(kind='line',x='step',y='pb',figsize=(20,10),ax=ax2)
ax2.set_ylabel("PB")
ax2.set_xlabel("Steps")

# 4th plot
df_params.plot(kind='line',x='step',y='pc',figsize=(20,10),ax=ax3)
ax3.set_ylabel("PC")
ax3.set_xlabel("Steps")

plt.show()

In [ ]:
tf_parameters = ['step', 'uBA','sbA','uBB','sbB','pA','pb','pc', 'pm','umax','nl',
                 'eLJ','uc','cost']
df_params = pd.DataFrame(columns = tf_parameters)

optimizer = tf.train.AdamOptimizer(2.e-4)

train = optimizer.minimize(cost)

init = tf.global_variables_initializer()

gradient_cost = optimizer.compute_gradients(cost)


with tf.Session() as sess:
    
    sess.run(init)
    ll = sess.run(cost)

    best_loss = ll
    best_ubA = sess.run(params['UBA'])
    best_sbA = sess.run(params['SIGBA'])
    best_ubB = sess.run(params['UBB'])
    best_sbB = sess.run(params['SIGBB'])
    best_pA = sess.run(params['PA'])
    best_pb = sess.run(params['PB'])
    best_pc = sess.run(params['PC'])
    best_nl = sess.run(params['NL'])
    best_elj = sess.run(params['E'])
    best_uc = sess.run(params['UC'])   
    for step in range(200):
        sess.run(train)
        ll = sess.run(cost)
        lubA = sess.run(params['UBA'])
        lsbA = sess.run(params['SIGBA'])
        lubB = sess.run(params['UBB'])
        lsbB = sess.run(params['SIGBB'])
        lpA =  sess.run(params['PA'])
        lpb = sess.run(params['PB'])
        lpc = sess.run(params['PC'])
        lnl = sess.run(params['NL'])
        lelj = sess.run(params['E'])
        luc = sess.run(params['UC'])                     
        if( ll < best_loss ):
            best_loss = ll
            best_ubA = lubA
            best_sbA = lsbA
            best_ubB = lubB
            best_sbB = lsbB
            best_pA = lpA
            best_pb = lpb
            best_pc = lpc
            best_nl = lnl
            best_elj = lelj
            best_uc = luc
      
        df_params = df_params.append({'step':step,'uBA':lubA*kT,'sbA':lsbA*kT,
                                           'uBB':lubB*kT,'sbB':lsbB*kT,'pA':lpA,'pb':lpb,
                                           'pc':lpc,'pm':1.0-pb-pc,'umax':umax,'nl':lnl,'eLJ':lelj*kT,
                                           'uc':luc*kT,'cost':ll},ignore_index=True)

    print("----- End of optimization --------");
    print("best", "x:", " uBA =", best_ubA*kT, "sbA =", best_sbA*kT, "uBB =", best_ubB*kT, "sbB =", best_sbB*kT, "pA =", best_pA, "pb = ", best_pb, "pc =", best_pc, "pm = ", 1.0-pb-pc, "umax = ", umax, "nl = ", best_nl, "eLJ = ", best_elj*kT, "uc = ", best_uc*kT, "cost =", best_loss)



In [ ]:
fig = plt.figure()

ax0 = fig.add_subplot(221)
ax1 = fig.add_subplot(222)
ax2 = fig.add_subplot(223)
ax3 = fig.add_subplot(224)

# 1st plot
df_params.plot(kind='line',x='step',y='cost', figsize=(20,10),ax=ax0)
ax0.set_ylabel("Cost")
ax0.set_xlabel("Steps")

# 2nd plot
df_params.plot(kind='line',x='step',y='nl',figsize=(20,10),ax=ax1)
ax1.set_ylabel("NL")
ax1.set_xlabel("Steps")

# 3rd plot
df_params.plot(kind='line',x='step',y='pb',figsize=(20,10),ax=ax2)
ax2.set_ylabel("PB")
ax2.set_xlabel("Steps")

# 4th plot
df_params.plot(kind='line',x='step',y='pc',figsize=(20,10),ax=ax3)
ax3.set_ylabel("PC")
ax3.set_xlabel("Steps")

plt.show()

In [ ]:
tf_parameters = ['step', 'uBA','sbA','uBB','sbB','pA','pb','pc', 'pm','umax','nl',
                 'eLJ','uc','cost']
df_params = pd.DataFrame(columns = tf_parameters)

var_list1 = [params['PC'],params['PB'],params['NL']]


## .minimize combines compute_gradients and apply_gradients

opt = tf.train.AdamOptimizer(learning_rate=2.e-4)
grads_and_vars = opt.compute_gradients(cost, var_list=var_list1)
nl_grad,_ = grads_and_vars[2]
train_op = opt.apply_gradients([grads_and_vars[0],grads_and_vars[1],(nl_grad*2.e5,params['NL'])])
######

#optimizer = tf.train.AdamOptimizer(2.e-4)
#train = optimizer.minimize(cost, var_list=var_list1)

init = tf.global_variables_initializer()

#gradient_cost = optimizer.compute_gradients(cost)


with tf.Session() as sess:
    
    sess.run(init)
    ll = sess.run(cost)

    best_loss = ll
    best_ubA = sess.run(params['UBA'])
    best_sbA = sess.run(params['SIGBA'])
    best_ubB = sess.run(params['UBB'])
    best_sbB = sess.run(params['SIGBB'])
    best_pA = sess.run(params['PA'])
    best_pb = sess.run(params['PB'])
    best_pc = sess.run(params['PC'])
    best_nl = sess.run(params['NL'])
    best_elj = sess.run(params['E'])
    best_uc = sess.run(params['UC'])   
    for step in range(300):
        sess.run(train_op)
        ll = sess.run(cost)
        lubA = sess.run(params['UBA'])
        lsbA = sess.run(params['SIGBA'])
        lubB = sess.run(params['UBB'])
        lsbB = sess.run(params['SIGBB'])
        lpA =  sess.run(params['PA'])
        lpb = sess.run(params['PB'])
        lpc = sess.run(params['PC'])
        lnl = sess.run(params['NL'])
        lelj = sess.run(params['E'])
        luc = sess.run(params['UC'])                     
        if( ll < best_loss ):
            best_loss = ll
            best_ubA = lubA
            best_sbA = lsbA
            best_ubB = lubB
            best_sbB = lsbB
            best_pA = lpA
            best_pb = lpb
            best_pc = lpc
            best_nl = lnl
            best_elj = lelj
            best_uc = luc
      
        df_params = df_params.append({'step':step,'uBA':lubA*kT,'sbA':lsbA*kT,
                                           'uBB':lubB*kT,'sbB':lsbB*kT,'pA':lpA,'pb':lpb,
                                           'pc':lpc,'pm':1.0-pb-pc,'umax':umax,'nl':lnl,'eLJ':lelj*kT,
                                           'uc':luc*kT,'cost':ll},ignore_index=True)

    print("----- End of optimization --------");
    print("best", "x:", " uBA =", best_ubA*kT, "sbA =", best_sbA*kT, "uBB =", best_ubB*kT, "sbB =", best_sbB*kT, "pA =", best_pA, "pb = ", best_pb, "pc =", best_pc, "pm = ", 1.0-pb-pc, "umax = ", umax, "nl = ", best_nl, "eLJ = ", best_elj*kT, "uc = ", best_uc*kT, "cost =", best_loss)




In [ ]:
fig = plt.figure()

ax0 = fig.add_subplot(221)
ax1 = fig.add_subplot(222)
ax2 = fig.add_subplot(223)
ax3 = fig.add_subplot(224)

# 1st plot
df_params.plot(kind='line',x='step',y='cost', figsize=(20,10),ax=ax0)
ax0.set_ylabel("Cost")
ax0.set_xlabel("Steps")

# 2nd plot
df_params.plot(kind='line',x='step',y='nl',figsize=(20,10),ax=ax1)
ax1.set_ylabel("NL")
ax1.set_xlabel("Steps")

# 3rd plot
df_params.plot(kind='line',x='step',y='pb',figsize=(20,10),ax=ax2)
ax2.set_ylabel("PB")
ax2.set_xlabel("Steps")

# 4th plot
df_params.plot(kind='line',x='step',y='pc',figsize=(20,10),ax=ax3)
ax3.set_ylabel("PC")
ax3.set_xlabel("Steps")

plt.show()